# manpower-planning.py

notebook 版本

## Manpower Planning Example
Source: http://www.gurobi.com/resources/examples/manpower-planning

This model is an example of a staffing problem. In staffing planning problems, choices must be made regarding the recruitment, training, layoffs (redundancy) and scheduling of staff. These problems are common across a broad range of both manufacturing and service industries.

此问题为 Staffing Problem。 需要选择雇佣、训练、冗余和调度员工。

In this example we’ll model and solve a manpower planning problem. We have three type of workers with different skills levels. For each year in the planning horizon the predicted number of required workers of each skill is given. It is possible to recruit new people, train workers to improve/decrease their skills or put them into part-time (short-time). The aim is to create an optimal multi-period operation plan to minimize the total number of layoffs over the whole horizon. An alternative aim is to minimize the total costs.

此例子中，我们有三种工人（不同技能）。每年预测需要不同级别的员工。可以雇佣新人，训练工人，或是作为兼职员工。目标是建立一个最优化最小代价的排班计划。

相关图书：

Note: you can download the model, implemented in Python, here. More information on this type of model can be found in the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams.

H. Paul Williams, Model Building in Mathematical Programming, fifth edition (Page 255-256, 354-356)

## Implementation with comments

First, we import the Gurobi Python Module and initialize the data structures with the given data.

In [1]:
from gurobipy import *

# tested with
#  Python 2.7.13 :: Anaconda 4.3.1 (x86_64) Gurobi 7.0.2
#  Python 2.7.6 & Gurobi 7.0.1

years = tuplelist(range(2+1))
skill_levels = [0, 1, 2]  # 0 = Unskilled, 1 = Semiskilled, 2 = Skilled
Unskilled = 0
Semiskilled = 1
Skilled = 2

CurrentStrength = [2000, 1500, 1000]
Requirement = [[1000, 1400, 1000],
               [500, 2000, 1500],
               [0, 2500, 2000]]
LeaveFirstYear = [0.25, 0.20, 0.10]
LeaveEachYear = [0.10, 0.05, 0.05]
ContinueFirstYear = [1 - a for a in LeaveFirstYear]
ContinueEachYear = [1 - a for a in LeaveEachYear]
LeaveDownGraded = 0.50
ContinueDownGraded = 1 - LeaveDownGraded
MaxRecruit = [500, 800, 500]
MaxRetrainUnskilled = 200
MaxOverManning = 150
MaxShortTimeWorking = 50
RetrainSemiSkilled = 0.25
ShortTimeUsage = 0.50

RetrainCost = [400, 500, 0]
RedundantCost = [200, 500, 500]
ShortTimeCost = [500, 400, 400]
OverManningCost = [1500, 2000, 3000]


### Create the recruit vars upper bound dictionary.

In [2]:
MaxRecruit2 = {(level, year) : MaxRecruit[level] for level in skill_levels for year in years}

### Next, we create a model and the variables. 

For each of the three skill levels and for each year we will create variables for the amount of workers that get recruited, put into part-time work, are available as workers, are redundant, are overmanned. For each pair of skill levels and each year we have a variable for the amount of workers that get retrained to a higher/lower skill level. The amount of people which are in part-time and can be recruited is limited.

In [3]:
model = Model('Manpower planning')

Recruit = model.addVars(skill_levels, years, ub= MaxRecruit2, name="Recruit")
ShortTime = model.addVars(skill_levels, years, ub=MaxShortTimeWorking,
                          name="ShortTime")
LaborForce = model.addVars(skill_levels, years, name="LaborForce")
Redundant = model.addVars(skill_levels, years, name="Redundant")
OverManned = model.addVars(skill_levels, years, name="OverManned")
Retrain = model.addVars(skill_levels, skill_levels, years, name="Retrain")

### Step 5: Add constraints

Next, we insert the constraints. The continuity constraints ensure that per skill level and per year the current needed workers (LaborForce) and the laidoff people and the people who gets retrained to the current level, minus the people who gets retrained from the current level to a different skills, equals the LaborForce of the last year (or the CurrentStrength in the first year) plus the recruited people. A certain amount of people leave the company each year, so this is also considered with a factor. This constraint describes the change in the total amount of employed workers.

In [4]:
model.addConstrs(
    (LaborForce[level, year] + Redundant[level, year] 
    - ContinueFirstYear[level] * Recruit[level, year]
    + quicksum(Retrain[level, level2, year]
               - ContinueEachYear[level] * Retrain[level2, level, year]
               for level2 in skill_levels if level2 < level)
    + quicksum(Retrain[level, level2, year]
               - 0.5 * Retrain[level2, level, year]
               for level2 in skill_levels if level2 > level)
    == ContinueEachYear[level] * (
        CurrentStrength[level] if year == years[0]
        else LaborForce[level, years[years.index(year)-1]])
    for year in years for level in skill_levels),
    "Continuity")

# RetainMaxUnskilled
model.addConstrs(
    (Retrain[Unskilled, Semiskilled, year] <= MaxRetrainUnskilled
    for year in years), "RetrainMaxUnskilled")
model.addConstrs(
    (Retrain[Unskilled, Skilled, year] <= 0 for year in years), "ForbidRetrainUnskilledToSkilled")

# RetrainingSemiSkilled
model.addConstrs(
    (Retrain[Semiskilled, Skilled, year] <=
     RetrainSemiSkilled * LaborForce[Skilled, year] for year in years), "RetrainingSemiSkilled")

## Overmanning
model.addConstrs(
    (OverManned.sum('*', year) <= MaxOverManning for year in years), "Overmanning")

# Requirements
model.addConstrs(
    (LaborForce[level, year] ==
     Requirement[year][level] +
     OverManned[level, year] +
     ShortTimeUsage * ShortTime[level, year]
    for year in years for level in skill_levels), "Requirements")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>}

### The first objective is to minimze the total number of laidoff workers. This can be stated as:

In [5]:
# Minimize TotalRedundantMen
obj = Redundant.sum()

# Minimize TotalCost
# obj = quicksum(
#     RetrainCost[level]*(Retrain[level, level+1, year] if level < 2 else 0)
#     + RedundantCost[level] * Redundant[level, year]
#     + ShortTimeCost[level] * ShortTime[level, year]
#     + OverManningCost[level] * OverManned[level, year]
#     for year in years
#     for level in skill_levels)

model.setObjective(obj)

### Step 6: Solve model

In [6]:
model.optimize()

Optimize a model with 30 rows, 72 columns and 117 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 2e+03]
Presolve removed 18 rows and 44 columns
Presolve time: 0.01s
Presolved: 12 rows, 28 columns, 56 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4000000e+02   5.187500e+02   0.000000e+00      0s
       8    8.4179688e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  8.417968750e+02


### Print variable values for optimal solution

In [7]:
# Display solution (print the name of each variable and the solution value)
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

Recruit[1,1] 649.303557
Recruit[1,2] 676.973684
Recruit[2,1] 500.000000
Recruit[2,2] 500.000000
ShortTime[0,0] 50.000000
ShortTime[0,1] 50.000000
ShortTime[0,2] 50.000000
ShortTime[1,0] 50.000000
ShortTime[2,0] 50.000000
LaborForce[0,0] 1157.031250
LaborForce[0,1] 675.000000
LaborForce[0,2] 175.000000
LaborForce[1,0] 1442.968750
LaborForce[1,1] 2000.000000
LaborForce[1,2] 2500.000000
LaborForce[2,0] 1025.000000
LaborForce[2,1] 1500.000000
LaborForce[2,2] 2000.000000
Redundant[0,0] 442.968750
Redundant[0,1] 166.328125
Redundant[0,2] 232.500000
OverManned[0,0] 132.031250
OverManned[0,1] 150.000000
OverManned[0,2] 150.000000
OverManned[1,0] 17.968750
Retrain[0,1,0] 200.000000
Retrain[0,1,1] 200.000000
Retrain[0,1,2] 200.000000
Retrain[1,2,0] 256.250000
Retrain[1,2,1] 80.263158
Retrain[1,2,2] 131.578947
Retrain[2,1,0] 168.437500
